In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession


In [4]:
spark=SparkSession.builder.config("spark-driver.host", "localhost").config("spark.driver.memory","4g") \
.config("spark.executor.memory","4g").appName("mr").getOrCreate()

In [5]:
spark

In [6]:
#import all required libraries

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline


In [7]:
# Initialize Spark session
spark = SparkSession.builder.appName("PopularBooksRecommendation").getOrCreate()

#Dataset Load
file_path = "C:/Users/psind/Downloads/Books_rating.csv/Books_rating.csv"

books_df = spark.read.csv("C:/Users/psind/Downloads/books_data.csv/books_data.csv", header=True, inferSchema=True)
rating_df = spark.read.csv("C:/Users/psind/Downloads/Books_rating.csv/Books_rating.csv", header=True, inferSchema=True)



# Merge the two DataFrames on 'Id'
df = rating_df.join(books_df, on="Title", how="inner")
df = spark.read.csv(file_path, header=True, inferSchema=True)

#columns required 
column = ['Id','User_id','review/score','Title']
df = df.select([col for col in column]).toDF(*column)
df = df.withColumn("review/score", df["review/score"].cast("float"))
df = df.na.drop("any")
df = df.dropDuplicates(subset=column)
df.show(5)


+----------+--------------+------------+--------------------+
|        Id|       User_id|review/score|               Title|
+----------+--------------+------------+--------------------+
|0918973031|A1PYZR7NRX62CN|         5.0|Dramatica for Scr...|
|0792391810| A29Z0B2L367ZO|         5.0|Vector Quantizati...|
|0195178548|A3A91IVPF679N3|         5.0|The Oxford Handbo...|
|0781810698|A1HHC466MCO9QE|         5.0|Beginner's Yoruba...|
|050552421X|A3I60ZNO274ZOL|         5.0|The Scarletti Cur...|
+----------+--------------+------------+--------------------+
only showing top 5 rows



In [8]:
#Filter the data based on the Book_id and User_id Limit thresholds

Id_limit = 200
User_id_limit = 10

Id_counts = df.groupBy("Id").count().filter(f"count >= {Id_limit}")
User_id_counts = df.groupBy("User_id").count().filter(f"count >= {User_id_limit}")

In [9]:
#Filtering and Index the dataframe to give the input the model

filtered_df = df.join(Id_counts, on="Id", how="inner") \
                .join(User_id_counts, on="User_id", how="inner") \
                .drop("count")
string_indexer_id = StringIndexer(inputCol="Id", outputCol="IdIndex")
string_indexer_user = StringIndexer(inputCol="User_id", outputCol="UserIndex")


pipeline = Pipeline(stages=[string_indexer_id, string_indexer_user])
new_df = pipeline.fit(filtered_df).transform(filtered_df)

In [10]:
#split the data into train and test 

(train_data, test_data) = new_df.randomSplit([0.8, 0.2], seed=42)


In [12]:
#ALS Model

als = ALS(maxIter=5, regParam=0.01, userCol="UserIndex", itemCol="IdIndex", ratingCol="review/score",coldStartStrategy="drop")
model = als.fit(train_data)



In [13]:
#predictions

predictions = model.transform(test_data)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="review/score", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 1.2380428861184438


In [27]:
#user recommendation

input_user_id = input("Enter the User ID: ")

# Find the corresponding UserIndex for the given User ID
user_index = None
for index, user_id in user_index_to_id.items():
    if user_id == input_user_id:
        user_index = index
        break

if user_index is not None:
    # Get the recommendations for the specific user
    user_recommend = model.recommendForAllUsers(5).filter(f"UserIndex == {user_index}").collect()

    if user_recommend:
        print(f"Recommendations for User: {input_user_id}")
        for rec in user_recommend[0].recommendations:
            book_id = rec.IdIndex
            book_title = book_id_to_title[book_id]
            print(f"- {book_title}")
    else:
        print(f"No recommendations found for User ID: {input_user_id}")
else:
    print(f"User ID '{input_user_id}' not found in the dataset.")

Enter the User ID:  A1N1YEMTI9DJ86


Recommendations for User: A1N1YEMTI9DJ86
- Ultrametabolism: The Simple Plan for Automatic Weight Loss
- Please Stop Laughing At Me...: One Woman's Inspirational Story
- Love & Respect: The Love She Most Desires; The Respect He Desperately Needs
- The Master Swing Trader: Tools and Techniques to Profit from Outstanding Short-Term Trading Opportunities
- Holy Bible New International Version
